# Probabilistic Programming - Assignment

Year: 2022-2023

In [1]:
# Enter your name and student ID
name = 
ID   = 

LoadError: syntax: incomplete: premature end of input

In this assignment, we will go through the Bayesian model design cycle:

 <img src="figures/modelling-procedure.png" alt="" width="600"> 

In questions 1 and 2, you will build a simple model, fit it to data and evaluate its performance on future data. You will find that its performance is not great. In question 3, you will improve the model in multiple ways. Finally, in question 4, you will do model selection based on free energy.

The final questions will require knowledge from the last probabilistic programming session. But questions 1 and 2 can be done relatively early in the course.

In [14]:
using Pkg
Pkg.activate("/root/work/")
Pkg.instantiate();

  Activating project at `~/work`


In [3]:
using CSV
using DataFrames
using LinearAlgebra
using ProgressMeter
using RxInfer
using Plots
default(label="",
        grid=false, 
        linewidth=3, 
        markersize=4,
        guidefontsize=12, 
        margins=15Plots.pt)

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1664

signal (2): Interrupt
in expression starting at none:0
ijl_array_grow_end at /cache/build/default-amdci4-6/julialang/julia-release-1-dot-8/src/array.c:955
ijl_array_ptr_1d_push at /cache/build/default-amdci4-6/julialang/julia-release-1-dot-8/src/array.c:1259
collect_backedges at /cache/build/default-amdci4-6/julialang/julia-release-1-dot-8/src/dump.c:1116 [inlined]
jl_collect_methcache_from_mod at /cache/build/default-amdci4-6/julialang/julia-release-1-dot-8/src/dump.c:1141
jl_typemap_node_visitor at /cache/build/default-amdci4-6/julialang/julia-release-1-dot-8/src/typemap.c:317 [inlined]
jl_typemap_visitor at /cache/build/default-amdci4-6/julialang/julia-release-1-dot-8/src/typemap.c:354 [inlined]
jl_typemap_array_visitor at /cache/build/default-amdci4-6/julialang/julia-release-1-dot-8/src/typemap.c:306
jl_typemap_visitor at /cache/build/default-amdci4-6/julialang/julia-release-1-dot-8/src/typema

LoadError: Failed to precompile CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b] to /root/.julia/compiled/v1.8/CSV/jl_g4jemW.

## Problem: Forecasting Air Quality

Many Europeans suspect that the air quality in their city is declining. A [recent study](https://doi.org/10.1016/j.snb.2007.09.060) measured the air quality of a major city in North Italy using an electronic nose. The measurements were made in the middle of the city and reflect urban activity. We will inspect the specific chemical concentrations found and build a model to accurately predict CO for future time points.

![https://www.theguardian.com/environment/2020/apr/07/air-pollution-linked-to-far-higher-covid-19-death-rates-study-finds](figures/air-milan-wide.png)

Photograph taken by Claudio Furlan/LaPresse/Zuma Press/Rex/Shutterstock ([link](https://www.theguardian.com/environment/2020/apr/07/air-pollution-linked-to-far-higher-covid-19-death-rates-study-finds))

The data can be found here: https://archive.ics.uci.edu/ml/datasets/Air+Quality. I've done some pre-processing and selected the most important features. In this assignment we will infer parameters in a model of the data and predict air quality in the future. For that purpose, the data has been split into past and future.

In [4]:
# Load training data
past_data = DataFrame(CSV.File("data/airquality_past.csv"))

LoadError: UndefVarError: CSV not defined

Let's visualize the carbon monoxide measurements over time.

In [5]:
scatter(past_data[:,1], 
        past_data[:,2], 
        size=(900,300), 
        color="black", 
        xlabel="time", 
        ylabel="CO (ppm)",
        ylims=[400,2000])

LoadError: UndefVarError: past_data not defined

## 1. Model specification & parameter estimation

We suspect that there is a temporal dependence in this dataset. In other words, the data changes relatively slowly over time and neighbouring data points end up being highly correlated. To exploit this correlation, we will build an _auto-regressive model_ of the form:

$$ y_k = \theta y_{k-1} + \epsilon_k \, , $$

where the noise $\epsilon_k$ is drawn from a zero-mean Gaussian with precision parameter $\tau$: 

$$ \epsilon_k \sim \mathcal{N}(0, \tau^{-1}) \, .$$

Tasks:
- [1pt] Specify the above equation as a probabilistic model in RxInfer, using $\tau = 1.0$.
- [1pt] Specify and execute an inference procedure to infer a posterior distribution for $\theta$.
- [1pt] Plot the inferred distribution over the interval $[0,\ 2]$.

In [6]:
### YOUR CODE HERE

## 2. Predictions & model evaluation

We want to evaluate the parameters inferred under the model. For now, we will do this by visually inspecting the 1-step ahead predictions on our data set. Later, we will use free energy as a metric. 

The posterior predictive distribution for the next time step is:

$$ p(y_{t+1} \mid y_{t}, \mathcal{D}) = \int p(y_{t+1} \mid \theta, y_{t}) p(\theta \mid \mathcal{D}) \, \mathrm{d}\theta \, , $$

where $\mathcal{D}$ refers to the data used to infer the posterior distribution. To make 1-step ahead predictions, you will have to loop over the data (i.e., `for t in 1:T`), plug in the current data point and compute the parameters of the posterior predictive distribution for the next data point. You may start from $t=2$, using $y_1$ as initial "previous observation". 

Tasks:
- [1pt] Derive the parameters of the posterior predictive and compute the 1-step ahead predictions on the data set.
- [1pt] Plot the first 10 predictions (posterior predictive variance in `ribbon=`) along with $y_{2:11}$ (scatterplot).

---

Note that if you failed to infer a posterior distribution in the previous question, you can still answer this question using a standard normal, $p(\theta) = \mathcal{N}(0,1)$.

In [7]:
### YOUR CODE HERE

## 3. Model critiqueing & improvement

From the results of the previous question, you may conclude that our initial model isn't great: it only considers extremely short-term changes, which are highly affected by noise. 

The model can be improved in two ways:
1. We set the noise level $\tau$ to $1.0$ but that was based on convenience, not on domain expertise or data. You are going to put a prior probability distribution over $\tau$ and infer a posterior $p(\tau \mid \mathcal{D})$ simultaneously. 
2. We can consider changes over longer periods of time by increasing the model order of the auto-regressive model. That corresponds to:

$$ y_k = \sum_{m=1}^{M} \theta_m y_{k-m} + \epsilon_k \, ,$$

where $M$ corresponds to model order.

Tasks:
- [1pt] Specify a higher-order AR model with an order parameter $M$, and with noise precision as a random variable.
- [1pt] Infer the approximate posteriors for $\theta$ and $\tau$, for model order $M=3$.
- [1pt] Visualize the posterior for the noise distribution $q(\tau)$ and the 1-step ahead predictions (mean and variance) on the data.

In [8]:
# Number of iterations of variational inference
n_iters = 10;

# Model order
M = 5;

In [9]:
### YOUR CODE HERE

## 4. Model selection & deployment

We now essentially have a different model for each value of $M$. Which is the best?

Tasks:
- [1pt] Compute the free energies for a given range of model orders and report the best performing one.
- [1pt] Deploy the best model (i.e., compute 1-step ahead predictions) on future data.

In [10]:
# Model order range
model_orders = [2,4,8,16,32];

In [11]:
### YOUR CODE HERE

In [12]:
# Load test data
future_data = DataFrame(CSV.File("data/airquality_future.csv"))

LoadError: UndefVarError: CSV not defined

In [13]:
### YOUR CODE HERE

## Submission

Before you submit the assignment, **make sure your notebook runs correctly**! You can do that by going to the `Kernel` tab in the toolbar and pressing `Restart & Run All`. 

This is important! If your code doesn't run, we can't verify the correctness of your answer.

When you're ready, head on over to Canvas and upload your notebook.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cf61448b-d960-4561-a62e-d8636c19b62c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>